In [1]:
import os
import sys
sys.path.insert(0, "../")

In [2]:
from core.nl_parser import SYSTEM, USER, NLParser

# Setup

In [3]:
open_ai_key = os.environ["OPENAI_API_KEY"]

In [4]:
nl_parser = NLParser(open_ai_key)

# Generate

In [7]:
duration = 90
focus = "Endurance"

In [8]:
nl_parser.generate(duration_min=duration, focus=focus)

Workout(name='Endurance Ride', focus='Endurance', steps=[Step(duration_s=300, pct_ftp=0.5, note=None, cadence=None, kind='warmup', pct_ftp_end=0.6), Step(duration_s=2400, pct_ftp=0.65, note=None, cadence=None, kind='steady', pct_ftp_end=None), Step(duration_s=600, pct_ftp=0.7, note=None, cadence=None, kind='steady', pct_ftp_end=None), Step(duration_s=2400, pct_ftp=0.65, note=None, cadence=None, kind='steady', pct_ftp_end=None), Step(duration_s=600, pct_ftp=0.7, note=None, cadence=None, kind='steady', pct_ftp_end=None), Step(duration_s=2400, pct_ftp=0.65, note=None, cadence=None, kind='steady', pct_ftp_end=None), Step(duration_s=300, pct_ftp=0.6, note=None, cadence=None, kind='cooldown', pct_ftp_end=0.5)])